In [14]:
import os
import sys
from snowflake.snowpark.types import StringType
from snowflake.snowpark.functions import udf
from snowflake.snowpark import Session
import pandas as pd
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)


In [15]:
# Unpack env vars
API_BASE = "http://localhost:8000/v1/completions"
SNOW_ROLE = os.environ["SNOW_ROLE"]
SNOW_WAREHOUSE = os.environ["SNOW_WAREHOUSE"]
SNOW_DATABASE = os.environ["SNOW_DATABASE"]
SNOW_SCHEMA = os.environ["SNOW_SCHEMA"]


# Establish snowpark session
def get_token():
    with open('/snowflake/session/token', 'r') as f:
        return f.read()

connection_params = {
    'host': os.environ['SNOWFLAKE_HOST'],
    'port': os.environ['SNOWFLAKE_PORT'],
    'protocol': 'https',
    'account': os.environ['SNOWFLAKE_ACCOUNT'],
    'authenticator': 'oauth',
    'token': get_token(),
    'role': SNOW_ROLE,
    'warehouse': SNOW_WAREHOUSE,
    'database': SNOW_DATABASE,
    'schema': SNOW_SCHEMA 
}


session = Session.builder.configs(connection_params).create()
token = get_token()



In [16]:
@udf(name="build_prompts", is_permanent=True, stage_location="@files", replace=True, session=session)
def build_prompt(name: str, feature_1: str, feature_2: str, target_customer1: str,feature_3: str) -> str:
    PROMPT_TEMPLATE = '''Here is a catchy product description for a {0} that highlights {1} and {2}. Personalize the description for {3} interested in {4}:'''
    return PROMPT_TEMPLATE.format(name, feature_1, feature_2, target_customer1,feature_3)

def get_df(table_name):
    df = session.table(table_name)
    df_new= df.select(df["id"],df["name"], build_prompt(df["name"], df["feature_1"], df["feature_2"], df["target_customer1"], df["feature_3"]).as_("prompt"))
    return df_new.select("id", "name", "prompt")


In [17]:
df_s = get_df("MISTRAL_VLLM_DB.PUBLIC.PRODUCT_INFO").to_pandas()
 
df_s.head()

,ID,NAME,PROMPT
0,A1234,EJOffice Standup Desk,Here is a catchy product description for a EJO...
1,A3214,EJOffice Chair,Here is a catchy product description for a EJO...
2,A3215,EJOffice Regular Desk,Here is a catchy product description for a EJO...
3,A3216,EJOffice Trash Wastebasket,Here is a catchy product description for a EJO...
4,A3217,EJOffice Paper Shredder,Here is a catchy product description for a EJO...


In [18]:
prompt_list=df_s["PROMPT"].values.tolist()

outputs = [client.completions.create(model="mistralai/Mistral-7B-Instruct-v0.2",
                                      prompt=p, 
                                      max_tokens=800,
                                      n=1,
                                      stop=None,
                                      temperature=0.9) for p in prompt_list]
descriptions = []
# Print the outputs.
for output in outputs:
        generated_text = output.choices[0].text
        descriptions.append(generated_text.strip())
        print(f"Generated text: {generated_text!r}")


Generated text: "\n\nIntroducing the EJOffice Standup Desk: Your Remote Office's Ergonomic and Affordable Upgrade! 🏠💼\n\n🌿 Made from elegant, sustainable real wood, the EJOffice Standup Desk is more than just a functional piece of furniture for your home office. It's an investment in your health, posture, and overall well-being as a remote employee.\n\n💪 Adjust the height to your preference for a customizable workstation that caters to your ergonomic needs. Stand, sit, or do both – the choice is yours! Say goodbye to the discomfort and health risks associated with extended sitting.\n\n💰 With its affordability, there's no reason not to upgrade your workspace. The EJOffice Standup Desk delivers unparalleled value, ensuring that you can work comfortably and efficiently – all while staying within your budget.\n\n🤝    Join the hundreds of proud remote office workers who've made the switch to the EJOffice Standup Desk. Start transforming your work experience today! 💻💪🌿"
Generated text: '\n\n

In [19]:
df_s['DESCRIPTION'] = descriptions
DESTINATION_TABLE = "PRODUCT_DESCRIPTIONS"
DESTINATION_COLUMNS = 'ID,NAME,DESCRIPTION'
 
_ = session.write_pandas(df_s[DESTINATION_COLUMNS.split(',')], 
                           DESTINATION_TABLE, 
                           auto_create_table=True, 
                           overwrite=True,
                           on_error='continue')